**AUTHOR:** <br>
Vasilis Raptis

**DATE:** <br>
21.05.2024 

**PURPOSE:** <br>
This notebook: 
- runs regenie step 1, using filted microarray genotypes and delirium phenotype file (eur, afr, amr)
- copy regenie step 1 output to bucket

**NOTES:** <br>
- use lists of filterd ids and snps from bucket, array_noY genotypes from workspace (created with 02_part1_genotypes_preprocessing.ipynb)
- afr population has >1M filtered snps, regenie throws an error. Use --force-step1
- run in background, using 00_run_notebook_in_background.ipynb
- running times :
 - amr: 5h, 15m, and 19s (with 4 CPUs & 15G RAM)
 - eur: 8.22h (with 16 CPUs, 14.4G RAM)
 - afr: 9.51h (with 4 CPUs & 15G RAM)
 
**UPDATE 09.07.24:**
- running times (with 16 CPUs, 14.4G RAM):
 - amr: 3.2h
 - eur: 7.9h
 - afr: 

**Setup:**

In [1]:
# libraries
library(data.table)
library(tidyverse)

## Get my bucket name
my_bucket  <- Sys.getenv("WORKSPACE_BUCKET")
## Google project name
GOOGLE_PROJECT <- Sys.getenv("GOOGLE_PROJECT")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubridate::second()  masks data.table::second()
✖ purrr::transpose()   masks data.table::transpose()
✖ lubridate::wday() 

In [3]:
# List data in my bucket folder
system(paste0("gsutil ls ", my_bucket, "/data/pheno/clean"), intern=T)
system(paste0("gsutil ls ", my_bucket, "/data/arrays/clean/all"), intern=T)
# List storage usage in workspace
system("du -h", intern=T)

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/afr_ids_clean.txt"  
[2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/afr_pheno_clean.txt"
[3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/amr_ids_clean.txt"  
[4] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/amr_pheno_clean.txt"
[5] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/eur_ids_clean.txt"  
[6] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/eur_pheno_clean.txt"
[7] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/1on10/"             
[8] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/pheno/clean/1on5/"

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.id"     
[2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_afr_clean.snplist"
[3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.id"     
[4] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_amr_clean.snplist"
[5] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.id"     
[6] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/arrays_qc_eur_clean.snplist"
[7] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/arrays/clean/all/logs/"

[1] "126G\t./microarray/plink_v7.1" "126G\t./microarray"           
[3] "2.1M\t./.ipynb_checkpoints"    "126G\t."

**Regenie Step 1:**

In [8]:
## create folder in workspace
system(paste0("mkdir -p regenie_out/step1/tmpdir"), intern=T)
system(paste0("ls regenie_out/step1/tmpdir"), intern=T)

character(0)

[1] "regenie_tmp_preds_l0_Y1"

In [4]:
## load pheno tables from bucket
system(paste0("mkdir -p ./pheno/"), intern=T)
system(paste0("gsutil cp ", my_bucket, "/data/pheno/clean/*pheno_clean.txt", " ./pheno/"), intern=T)
system(paste0("ls ./pheno/"), intern=T)
## load filtered lists from bucket
system(paste0("gsutil cp ", my_bucket, "/data/arrays/clean/all/arrays_qc_*.snplist", " ./microarray/plink_v7.1/"), intern=T)
system(paste0("gsutil cp ", my_bucket, "/data/arrays/clean/all/arrays_qc_*.id", " ./microarray/plink_v7.1/"), intern=T)
system(paste0("ls ./microarray/plink_v7.1/"), intern=T)

character(0)

character(0)

[1] "afr_pheno_clean.txt" "amr_pheno_clean.txt" "eur_pheno_clean.txt"

character(0)

character(0)

[1] "arrays_noY.bed"              "arrays_noY.bim"             
 [3] "arrays_noY.fam"              "arrays_noY.log"             
 [5] "arrays_qc_afr_clean.id"      "arrays_qc_afr_clean.log"    
 [7] "arrays_qc_afr_clean.snplist" "arrays_qc_amr_clean.id"     
 [9] "arrays_qc_amr_clean.log"     "arrays_qc_amr_clean.snplist"
[11] "arrays_qc_eur_clean.id"      "arrays_qc_eur_clean.log"    
[13] "arrays_qc_eur_clean.snplist"

In [6]:
system(paste0("mkdir -p ./regenie_out/step1"), intern=T)


character(0)

American set:

In [9]:
# paths
bed     = "microarray/plink_v7.1/arrays_noY"
keep    = "microarray/plink_v7.1/arrays_qc_amr_clean.id"
extract = "microarray/plink_v7.1/arrays_qc_amr_clean.snplist"
pheno   = "pheno/amr_pheno_clean.txt"
out     = "regenie_out/step1/del_amr_clean_step1"

system(paste0("regenie --step 1 ",
              " --bed ", bed,
              " --keep ", keep,
              " --extract ", extract,
              " --phenoFile ", pheno,
              " --phenoCol delirium_status ",
              " --covarFile ", pheno,
              " --covarColList age,PC{1:10} ",
              " --catCovarList sex ",
              " --bt ", 
              " --bsize 1000 ", 
              " --lowmem --lowmem-prefix regenie_out/step1/tmpdir/regenie_tmp_preds ",
              " --threads $(nproc) ",
              " --out ", out
             ),
       intern=T)

European set:

In [10]:
# paths
bed     = "microarray/plink_v7.1/arrays_noY"
keep    = "microarray/plink_v7.1/arrays_qc_eur_clean.id"
extract = "microarray/plink_v7.1/arrays_qc_eur_clean.snplist"
pheno   = "pheno/eur_pheno_clean.txt"
out     = "regenie_out/step1/del_eur_clean_step1"

system(paste0("regenie --step 1 ",
              " --bed ", bed,
              " --keep ", keep,
              " --extract ", extract,
              " --phenoFile ", pheno,
              " --phenoCol delirium_status ",
              " --covarFile ", pheno,
              " --covarColList age,PC{1:10} ",
              " --catCovarList sex ",
              " --bt ", 
              " --bsize 1000 ", 
              " --lowmem --lowmem-prefix regenie_out/step1/tmpdir/regenie_tmp_preds ",
              " --threads $(nproc) ",
              " --out ", out
             ),
       intern=T)

African set:

In [11]:
# paths
bed     = "microarray/plink_v7.1/arrays_noY"
keep    = "microarray/plink_v7.1/arrays_qc_afr_clean.id"
extract = "microarray/plink_v7.1/arrays_qc_afr_clean.snplist"
pheno   = "pheno/afr_pheno_clean.txt"
out     = "regenie_out/step1/del_afr_clean_step1"

system(paste0("regenie --step 1 ",
              "--force-step1 ",
              " --bed ", bed,
              " --keep ", keep,
              " --extract ", extract,
              " --phenoFile ", pheno,
              " --phenoCol delirium_status ",
              " --covarFile ", pheno,
              " --covarColList age,PC{1:10} ",
              " --catCovarList sex ",
              " --bt ", 
              " --bsize 1000 ", 
              " --lowmem --lowmem-prefix regenie_out/step1/tmpdir/regenie_tmp_preds ",
              " --threads $(nproc) ",
              " --out ", out
             ),
       intern=T)

**Copy to bucket:**

In [7]:
# step1 loco predictions
system(paste0("gsutil cp regenie_out/step1/del_*_step1_1.loco", " ", my_bucket, "/data/regenie/step1/"), intern=T)
# path file (modify if reused in another location)
system(paste0("gsutil cp regenie_out/step1/del_*_step1_pred.list", " ", my_bucket, "/data/regenie/step1/"), intern=T)
# log files
#system(paste0("gsutil cp regenie_out/step1/del_*_step1.log", " ", my_bucket, "/data/regenie/step1/"), intern=T)
# check
system(paste0("gsutil ls ", my_bucket, "/data/regenie/step1/"), intern=T)

character(0)

character(0)

[1] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_afr_clean_1on10_step1_1.loco"   
 [2] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_afr_clean_1on10_step1_pred.list"
 [3] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_afr_step1_1.loco"               
 [4] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_afr_step1_pred.list"            
 [5] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_amr_clean_1on10_step1_1.loco"   
 [6] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_amr_clean_1on10_step1_pred.list"
 [7] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_amr_step1_1.loco"               
 [8] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_amr_step1_pred.list"            
 [9] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_eur_clean_1on10_step1_1.loco"   
[10] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_eur_clean_1on10_step1_pred.list"
[11] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_eur_clean_1on5_step1_1.loco"    
[12] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_eur_clean_1on5_step1_pred.list" 
[13] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_eur_step1_1.loco"               
[14] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/del_eur_step1_pred.list"            
[15] "gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/data/regenie/step1/logs/"